# imports

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# random shit

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_id = "codellama/CodeLlama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir='models')
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    cache_dir="models/"
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.12it/s]


In [4]:
model.to('cuda')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [7]:
prompt = '''def remove_non_ascii(s: str) -> str:
    """ <FILL_ME>
    return result
'''

tokenized = tokenizer(prompt, return_tensors="pt")
input_ids = tokenized["input_ids"].to("cuda")
attention_mask = tokenized["attention_mask"].to("cuda")
output = model.generate(
    input_ids,
    max_new_tokens=200,
    attention_mask=attention_mask,
    pad_token_id=tokenizer.eos_token_id
)
output = output[0].to("cpu")

filling = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)
print(prompt.replace("<FILL_ME>", filling))

def remove_non_ascii(s: str) -> str:
    """ Remove non-ASCII characters from a string.

    Args:
        s: The string to remove non-ASCII characters from.

    Returns:
        The string with non-ASCII characters removed.
    """
    result = ""
    for c in s:
        if ord(c) < 128:
            result += c
    return result



In [17]:
prompt = '''def remove_non_ascii(s: str) -> str:
    """ <FILL_ME>
    return result
'''

tkns_tmp = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")
tkns_tmp

tensor([[    1, 32007,   822,  3349, 29918,  5464, 29918,   294, 18869, 29898,
         29879, 29901,   851, 29897,  1599,   851, 29901,    13,  1678,  9995,
         29871, 32008,    13,  1678,   736,  1121,    13, 32009]],
       device='cuda:0')

In [23]:
for ti in tkns_tmp:
    token = tokenizer.convert_ids_to_tokens(ti)
    print(f"ID: {ti}, Token: {token}")

ID: tensor([    1, 32007,   822,  3349, 29918,  5464, 29918,   294, 18869, 29898,
        29879, 29901,   851, 29897,  1599,   851, 29901,    13,  1678,  9995,
        29871, 32008,    13,  1678,   736,  1121,    13, 32009],
       device='cuda:0'), Token: ['<s>', '▁<PRE>', '▁def', '▁remove', '_', 'non', '_', 'as', 'cii', '(', 's', ':', '▁str', ')', '▁->', '▁str', ':', '<0x0A>', '▁▁▁', '▁"""', '▁', '▁<SUF>', '<0x0A>', '▁▁▁', '▁return', '▁result', '<0x0A>', '▁<MID>']


In [7]:
tokenizer(prompt, return_tensors="pt")

{'input_ids': tensor([[    1, 32007,   822,  3349, 29918,  5464, 29918,   294, 18869, 29898,
         29879, 29901,   851, 29897,  1599,   851, 29901,    13,  1678,  9995,
         29871, 32008,    13,  1678,   736,  1121,    13, 32009]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}

In [9]:
tokenized.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
token_pre, token_mid, token_suf, token_eot = tokenizer.special_tokens_map['additional_special_tokens']

['▁<PRE>', '▁<MID>', '▁<SUF>', '▁<EOT>']

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'additional_special_tokens': ['▁<PRE>', '▁<MID>', '▁<SUF>', '▁<EOT>']}

In [36]:
tokenizer.convert_tokens_to_ids(['▁<MID>'])

[32009]

In [ ]:
tokenizer.all_special_ids

[1, 2, 0, 32007, 32009, 32008, 32010]

In [40]:
[(token, idx) for token, idx in tokenizer.vocab.items() if not any(c.isalnum() for c in token)]

[('▁}', 500),
 ('^-', 21583),
 ('▁);', 3482),
 (':@"', 8593),
 ('}}}\\', 20388),
 ('===', 25512),
 ('****', 2328),
 ('================', 9166),
 ('?', 29973),
 ('>,', 10202),
 ('}\\,', 15379),
 ('▁%>%', 12437),
 ('}"', 5038),
 ('(...)', 29077),
 ('《', 30866),
 ('�', 30140),
 ("'))", 8785),
 ('////////', 10797),
 ("▁'@", 18803),
 ('|}{', 29632),
 ('!!!', 21004),
 ('"))', 5783),
 (')--(', 25062),
 (';&', 25359),
 ('$),', 25942),
 ('{:', 25641),
 ('`](', 7072),
 ('▁–', 785),
 ("▁'';", 18231),
 ('▒', 30686),
 (':\\', 3583),
 (';</', 14814),
 ('►', 30318),
 ('∷', 31766),
 ('▁[_', 23160),
 ('∥', 30876),
 ('}_{\\', 6353),
 ('}{(', 19048),
 ('(-', 6278),
 ('▁•', 10266),
 ('?.', 9808),
 (')', 29897),
 (').', 467),
 ('{{\\', 16195),
 ("':'", 22099),
 ('▁/\\', 18826),
 ('▁§', 16683),
 ('____', 7652),
 (')})', 26972),
 ('▁}\\', 17704),
 ('="{', 10724),
 ('▁$(\\', 12211),
 ('̪', 31317),
 ('™', 30536),
 ('`}', 26357),
 ('}$,', 4429),
 ('/)', 4551),
 ('}{\\', 3331),
 ('▁($\\', 20430),
 ("'$", 13090),

['<s>', '</s>', '<unk>', '▁<PRE>', '▁<MID>', '▁<SUF>', '▁<EOT>']